In [9]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
from random import randint
from keras.models import model_from_yaml


In [10]:
abba = pd.read_csv('abba.csv')
acdc = pd.read_csv('acdc.csv')

abba_corpus = ' ' 
for index, row in abba.iterrows():
    abba_corpus = abba_corpus + str(row['lyrics'])

acdc_corpus = ' ' 
for index, row in acdc.iterrows():
    acdc_corpus = acdc_corpus + str(row['lyrics'])
    
abba_corpus = abba_corpus.lower()
acdc_corpus = acdc_corpus.lower()

corpus = abba_corpus + acdc_corpus

# Make our encoders annd decoders 
corpus_length = len(corpus)
chars = sorted(list(set(corpus)))
sentence_length = 20
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

In [11]:
from keras.models import model_from_json

# Build our network from loaded architecture and weights
with open('model.json') as model_file:
    architecture = model_file.read()

model = model_from_json(architecture)
model.load_weights('weights-05.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 64)            35072     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 72)                4680      
_________________________________________________________________
activation (Activation)      (None, 72)                0         
Total params: 72,776
Trainable params: 72,776
Non-trainable params: 0
_________________________________________________________________


In [12]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [13]:
def generate(seed_pattern):
        X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
        #print(X.shape)
        for i, character in enumerate(seed_pattern):
            X[0, i, encoding[character]] = 1
        
        generated_text = ""
        for i in range(500):
            # even de temperatuur toevoegen.
            prediction = sample(model.predict(X, verbose=0)[0],0.3)
            generated_text += decoding[prediction]

            activations = np.zeros((1, 1, num_chars), dtype=np.bool)
            activations[0, 0, prediction] = 1
            X = np.concatenate((X[:, 1:, :], activations), axis=1)

        return generated_text

def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = abba_corpus[seed:seed + sentence_length]

        return pattern

In [14]:

seed = make_seed('in the bard and show you on your lovelight')
print(seed)
txt =  generate(seed)
print(txt)

in the bard and show

i want to the say and the show
i'm a man it in the sing
i can't me feel a put the way
somether
she's gonna be the said
i could the stay the say
i have it of the say
she's got the gonna like a better
i want to she can your like it to me
when you know i can't got it alound the sun she was on the pack
i'm gonna shooting to have a fries
we all the gonna make a canger of a light
like a be can come in the show
i'm a brow i can't got the fire
i'm gonna cander the say
i want the startin


## Sample song
In the bard and show you on your lovelight and i can't get the mowner i'm a marion an and every mind, there's a boot